In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [ ]:

file_s = "./US-all/US_confirmed.csv"
with open(file_s, newline='') as csvfile:
    data = np.array(np.genfromtxt(csvfile, delimiter=','))

print(type(data), data.shape)
confirmed_M = data[1:, 1:]
# this way, all no number data is convert to nan
# (52, 439), 51 regions and 438 days data

file_s = "./US-all/US_deaths.csv"
with open(file_s, newline='') as csvfile:
    data = np.array(np.genfromtxt(csvfile, delimiter=','))

print(type(data), data.shape)
deaths_M = data[1:, 1:]

In [ ]:

# NY row 34
days_A  = np.arange(deaths_M.shape[1])
# NY_confirm = confirmed_M[34]
# NY_deaths  = deaths_M   [34]

# plt.plot(days_A, NY_confirm)
# plt.plot(days_A, NY_deaths )

In [ ]:

## in order to generate x date labels
from datetime import date

date0000 = date.fromisoformat("2020-01-22")
# date2002 = date.fromisoformat("2020-02-01")
# delta = date2002-date0000
# print(delta.days)

month_str = ["Empty", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
xlabels = month_str[2:12+1] + month_str[1:4+1]
print(len(xlabels))

idxs = []
for month in range(2, 12+1):
    date_i = date.fromisoformat("2020-%02d-01"%(month))
    delta_i = date_i - date0000
    idxs.append(delta_i.days)
for month in range(1, 4+1):
    date_i = date.fromisoformat("2021-%02d-01"%(month))
    delta_i = date_i - date0000
    idxs.append(delta_i.days)
print(len(idxs))

In [ ]:

## plot US national confirmed case and deaths over days
US_confirm = np.sum(confirmed_M, axis=0)
US_deaths  = np.sum(deaths_M, axis=0)

US_confirm_dt = US_confirm[1:] - US_confirm[0:-1]
US_confirm_dt = np.append(US_confirm_dt, US_confirm_dt[-1])
US_deaths_dt  = US_deaths[1:]  - US_deaths[0:-1]
US_deaths_dt  = np.append(US_deaths_dt, US_deaths_dt[-1])
print(US_confirm_dt.shape, US_deaths_dt.shape)

ax = plt.subplot(1, 2, 1)
ax.plot(days_A, US_confirm, "tab:blue", label="total")
ax.set_ylabel("confirmed cases")
ax.set_xlabel("days between 20200122~20210403")
ax.legend()
ax.grid(True)
ax2 = ax.twinx()
ax2.plot(days_A, US_confirm_dt, "tab:red", label="daily")
ax2.set_ylabel("daily new confirmed cases")
ax2.set_ylim(0, 350000*2)
ax2.legend()
temp = ["0", "100K", "200K", "300K", "400K", "500K", "600K"]
plt.yticks(np.arange(0, 350000*2, 100000), temp)
plt.xticks(idxs, xlabels)
# ax.grid(True)
# ax2.grid(True)
plt.title("US Covid19 confirmed graph")

ax = plt.subplot(1, 2, 2)
ax.plot(days_A, US_deaths, "tab:blue", label="total")
ax.grid(True)
ax.legend()
ax.set_ylabel("deaths")
temp = ["0", "100K", "200K", "300K", "400K", "500K"]
plt.yticks(np.arange(0, 600000, 100000), temp )
ax2 = ax.twinx()
ax2.plot(days_A, US_deaths_dt, "tab:red", label="daily")
ax2.set_ylabel("daily deaths")
ax2.set_ylim(0, 5000*2)
ax2.legend()
plt.xlabel("days between 20200122~20210403")
plt.xticks(idxs, xlabels)
plt.title("US Covid19 death graph")

In [ ]:

# wenchen's reference 0
# does vaccine roll change the curve ?
print(US_confirm.shape, US_deaths.shape) # 438
print(days_A.shape)                      # 438

date200122 = date.fromisoformat("2020-01-22")
date201105 = date.fromisoformat("2020-11-05")
date210128 = date.fromisoformat("2021-01-28")
date210215 = date.fromisoformat("2021-02-15")
date210403 = date.fromisoformat("2021-04-03")
days_i01   = (date201105 - date200122).days
days_i02   = (date210128 - date200122).days
days_i02_1 = (date210215 - date200122).days
days_i03   = (date210403 - date200122).days

In [ ]:
# fit poly
x01 = np.arange(days_i01, days_i02+1)
y01 = US_confirm[days_i01:days_i02+1]
y01_d = US_deaths[days_i01:days_i02+1]

x02 = np.arange(days_i01, days_i03+1)
y02 = US_confirm[days_i01:days_i03+1]
y02_d =US_deaths[days_i01:days_i03+1]

x02_1 = np.arange(days_i01, days_i02_1+1)
y02_1 = US_confirm[days_i01:days_i02_1+1]
y02_1d = US_deaths[days_i01:days_i02_1+1]
print(x.shape, y.shape)
params01 = np.polyfit(x01, y01, 2)
curvef01 = np.poly1d(params01)
params01_d = np.polyfit(x01, y01_d, 2)
curvef01_d = np.poly1d(params01_d)
params02 = np.polyfit(x02, y02, 2)
curvef02 = np.poly1d(params02)
params02_d = np.polyfit(x02, y02_d, 2)
curvef02_d = np.poly1d(params02_d)
# x03 = np.arange(0, days_i03+1)
# y03 = US_confirm[0:days_i03+1]
# params03 = np.polyfit(x03, y03, 2)
# curve_f03 = np.poly1d(params03)

In [ ]:
## KS-test if data fit curve
def KS_test(x, y, curvef, name, thres=0.05):
    curve_y = curvef(x)
    KS_test = np.max(np.abs(curve_y - y))/ (np.max(y)-np.min(y))
    print(name, "KS_test", KS_test)
    if KS_test < thres:
        print("{}: accept H0, data fits curve under {} error margin".format(name, thres))
    else:
        print("{}: reject H0, data doesn't fit curve under {} error margin".format(name, thres))

KS_test(x01, y01, curvef01, "test01")
# KS_test(x02, y02, curvef01, "test02")
KS_test(x02_1, y02_1, curvef01, "test02")
KS_test(x01, y01_d, curvef01_d, "test01_d")
KS_test(x02_1, y02_1d, curvef01_d, "test02_d")
# In [220]: %paste -q
# test01 KS_test 0.033403327886845094
# test01: accept H0, data fits curve under 0.05 error margin
# test02 KS_test 0.16471545719748104
# test02: reject H0, data doesn't fit curve under 0.05 error margin
# test01_d KS_test 0.016404002776755685
# test01_d: accept H0, data fits curve under 0.05 error margin
# test02_d KS_test 0.06910858240500009
# test02_d: reject H0, data doesn't fit curve under 0.05 error margin

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(days_A, US_confirm, linewidth=3, label="confirmed cases")
plt.plot(x02, curvef01(x02), "--", label="curve fit Nov20~Jan21")
plt.plot(x02, curvef02(x02), "--", label="curve fit Nov20~Mar21")
plt.ylabel("confirmed cases")
plt.xticks(idxs, xlabels)
plt.grid(True)
plt.legend()
plt.title("US Covid19 confirmed graph")
plt.subplot(1, 2, 2)
plt.plot(days_A, US_deaths, linewidth=3, label="deaths")
plt.plot(x02, curvef01_d(x02), "--", label="curve fit Nov20~Jan21")
plt.plot(x02, curvef02_d(x02), "--", label="curve fit Nov20~Mar21")
plt.ylabel("deaths")
plt.xticks(idxs, xlabels)
plt.grid(True)
plt.legend()
plt.title("US Covid19 deaths graph")

In [ ]:

num_regions, num_days = confirmed_M.shape[0], confirmed_M.shape[1]

## read our external data
# ..............................................................................
file_s = "./US-all/grocery.csv"
with open(file_s, newline='') as csvfile:
    data = np.array(np.genfromtxt(csvfile, delimiter=','))
print(type(data), data.shape)
# (52, 415), 51 regions and 414 days data, 24 days lacking data at the beginning
# (52, 439), 51 regions and 438 days data, deaths and confirm data
# 20200215~20210403

US_grocery_M = data[1:, 1:]
US_grocery_A = np.sum(US_grocery_M, axis=0)
US_grocery_A01 = np.append(np.array([0]*24), US_grocery_A) / num_regions
print(US_grocery_A01.shape)

# ..............................................................................
file_s = "./US-all/park.csv"
with open(file_s, newline='') as csvfile:
    data = np.array(np.genfromtxt(csvfile, delimiter=','))
print(type(data), data.shape)

US_park_M = data[1:, 1:]
US_park_A = np.sum(US_park_M, axis=0)
US_park_A01 = np.append(np.array([0]*24), US_park_A) / num_regions
print(US_park_A01.shape)

# ..............................................................................
file_s = "./US-all/resident.csv"
with open(file_s, newline='') as csvfile:
    data = np.array(np.genfromtxt(csvfile, delimiter=','))
print(type(data), data.shape)

US_resident_M = data[1:, 1:]
US_resident_A = np.sum(US_resident_M, axis=0)
US_resident_A01 = np.append(np.array([0]*24), US_resident_A) / num_regions
print(US_resident_A01.shape)

# ..............................................................................
file_s = "./US-all/retail.csv"
with open(file_s, newline='') as csvfile:
    data = np.array(np.genfromtxt(csvfile, delimiter=','))
print(type(data), data.shape)

US_retail_M = data[1:, 1:]
US_retail_A = np.sum(US_retail_M, axis=0)
US_retail_A01 = np.append(np.array([0]*24), US_retail_A) / num_regions
print(US_retail_A01.shape)

# ..............................................................................
file_s = "./US-all/workplace.csv"
with open(file_s, newline='') as csvfile:
    data = np.array(np.genfromtxt(csvfile, delimiter=','))
print(type(data), data.shape)

US_work_M = data[1:, 1:]
US_work_A = np.sum(US_work_M, axis=0)
US_work_A01 = np.append(np.array([0]*24), US_work_A) / num_regions
print(US_work_A01.shape)

In [ ]:
## plot grocery and retail
days_A = days_A
zeros_A = np.zeros_like(days_A)

plt.plot(days_A, zeros_A)        # base line
plt.plot(days_A, US_grocery_A01, label="grocery")
plt.plot(days_A, US_retail_A01, label="retail")
plt.plot(days_A, US_confirm/np.max(US_confirm)*100, label="US_confirm scaled") # scaled and plot on the same graph
plt.plot(days_A, US_deaths/np.max(US_deaths)*100,   label="US_deaths scaled") # scaled and plot on the same graph
plt.legend()
plt.xticks(idxs, xlabels)
plt.grid(True)
plt.ylabel("change percent compared to baseline")
plt.title("grocery, retail and covid data")

In [ ]:
## plot park
plt.plot(days_A, zeros_A)        # base line
plt.plot(days_A, US_park_A01, label="park")
plt.plot(days_A, US_confirm/np.max(US_confirm)*100, label="US_confirm scaled") # scaled and plot on the same graph
plt.plot(days_A, US_deaths/np.max(US_deaths)*100,   label="US_deaths scaled") # scaled and plot on the same graph
plt.legend()
plt.xticks(idxs, xlabels)
plt.grid(True)
plt.ylabel("change percent compared to baseline")
plt.title("park and covid data")

In [ ]:
## plot workplace and resident
plt.plot(days_A, zeros_A)        # base line
plt.plot(days_A, US_work_A01, label="workplace")
plt.plot(days_A, US_resident_A01, label="resident")
plt.plot(days_A, US_confirm/np.max(US_confirm)*100, label="US_confirm scaled") # scaled and plot on the same graph
plt.plot(days_A, US_deaths/np.max(US_deaths)*100,   label="US_deaths scaled") # scaled and plot on the same graph
plt.legend()
plt.xticks(idxs, xlabels)
plt.grid(True)
plt.ylabel("change percent compared to baseline")
plt.title("workplace, resident and covid data")

In [ ]:

## -----------------------------------------------------------------------------
## Does park closure changed daily new confirmed number?
print(US_park_A01.shape, days_A.shape, US_confirm_dt.shape)

date200122 = date.fromisoformat("2020-01-22")
date200701 = date.fromisoformat("2020-07-01")
date200815 = date.fromisoformat("2020-08-15")
date200930 = date.fromisoformat("2020-09-30")
days_i01   = (date200701 - date200122).days
days_i02   = (date200815 - date200122).days
days_i03   = (date200930 - date200122).days

In [ ]:
bfore_data = US_confirm_dt[days_i01: days_i02+1]
after_data = US_confirm_dt[days_i02: days_i03]
print(bfore_data.shape, after_data.shape)

## Z test
X = bfore_data - after_data
Z = np.mean(X) / np.sqrt(np.var(X))
print(Z) # 1.3932069618786034
# take significance (1-alpha) = 5%, two-tail test, normal(1.96) = (1-2.5%)
if Z > 1.96 or Z < -1.96:
    print("reject H0, two samples have significantly different mean values")
else:
    print("accept H0, two samples don't significantly different mean values")